In [2]:
import pandas as pd
import numpy as np
import itertools
import sklearn.metrics as met
import xgboost as xgb

In [3]:
training = pd.read_csv('../../csv/chat_based_features_training.csv')
test = pd.read_csv('../../csv/chat_based_features_test.csv')

In [4]:
#training_tf_idf = pd.read_csv('../../dataset/csv/tf_idf_train.csv')

In [5]:
column_names = training.columns.values.tolist()[1:-1]
print column_names

['number of conversation', 'percent of conversations started by the author', 'difference between two preceding lines[s]', 'number of messages sent', 'average percent of lines in conversation', 'average percent of characters in conversation', 'number of characters sent by the author', 'mean time of messages sent']


In [6]:
training.tail()

,autor,number of conversation,percent of conversations started by the author,difference between two preceding lines[s],number of messages sent,average percent of lines in conversation,average percent of characters in conversation,number of characters sent by the author,mean time of messages sent,is sexual predator
90004,1892883f0abcc6a4ba158fc745aa8538,1,0.000000,0.000000,4,0.800000,0.884615,23,68640,0
90005,ab7ced9657817b971afb5e21da60a35d,1,0.000000,0.000000,1,0.250000,0.588235,50,82320,0
90006,39bb57ea865ba830ce16c0117e4f19ee,1,0.000000,60.000000,2,0.500000,0.818182,27,78060,0
90007,bc724681935f49bbcd8156cfb716d7ce,3,0.666667,234.333333,53,0.490718,0.463103,1194,82620,0
90008,fa622ac35fec508c9dcbd336885b18a0,1,0.000000,0.000000,2,0.400000,0.384615,5,1140,0


In [7]:
# Remove scale_post_weight -> smaller F1 score, lower recall but increase precision
# Remove gama for bigger precision
sex_offender0 = xgb.XGBClassifier(max_depth=25, n_estimators=300, learning_rate=0.007, scale_pos_weight=5, gamma=7)
sex_offender1 = xgb.XGBClassifier(max_depth=25, n_estimators=300)
sex_offender2 = xgb.XGBClassifier(max_depth=5, n_estimators=220, learning_rate=0.007, scale_pos_weight=20, gamma=2)

In [8]:
sex_offender0.fit(training[['number of conversation', 'percent of conversations started by the author', 'number of messages sent', 'average percent of lines in conversation', 'number of characters sent by the author']], training.iloc[:,[9]])
sex_offender1.fit(training[['number of conversation', 'percent of conversations started by the author', 'number of messages sent', 'average percent of lines in conversation', 'number of characters sent by the author']], training.iloc[:,[9]])
sex_offender2.fit(training[['number of conversation', 'percent of conversations started by the author', 'number of messages sent', 'average percent of lines in conversation', 'number of characters sent by the author']], training.iloc[:,[9]])

/home/weenkus/anaconda3/envs/py27/lib/python2.7/site-packages/sklearn/preprocessing/label.py:108: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/weenkus/anaconda3/envs/py27/lib/python2.7/site-packages/sklearn/preprocessing/label.py:108: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/weenkus/anaconda3/envs/py27/lib/python2.7/site-packages/sklearn/preprocessing/label.py:108: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=2, learning_rate=0.007, max_delta_step=0, max_depth=5,
       min_child_weight=1, missing=None, n_estimators=220, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=20, seed=0, silent=True, subsample=1)

In [9]:
prediction0 = sex_offender0.predict(test[['number of conversation', 'percent of conversations started by the author', 'number of messages sent', 'average percent of lines in conversation', 'number of characters sent by the author']])
prediction1 = sex_offender1.predict(test[['number of conversation', 'percent of conversations started by the author', 'number of messages sent', 'average percent of lines in conversation', 'number of characters sent by the author']])
prediction2 = sex_offender2.predict(test[['number of conversation', 'percent of conversations started by the author', 'number of messages sent', 'average percent of lines in conversation', 'number of characters sent by the author']])

In [24]:
def bagging(vote_number, prediction_list):
    total_prediction = []
    for i in range(len(prediction_list[0])):
        voters = 0
        
        for prediction in prediction_list:
            if prediction[i] == 1:
                voters += 1
                
        if voters >= vote_number:
            total_prediction.append(1)
        else:
            total_prediction.append(0)
                
    return np.array(total_prediction)

prediction = bagging(2, [prediction0, prediction1, prediction2])

In [25]:
print 'Accuracy: ', met.accuracy_score(test[['is sexual predator']], prediction) 
print 'Precision: ', met.precision_score(test[['is sexual predator']], prediction)
print 'Recall:', met.recall_score(test[['is sexual predator']], prediction)
print 'F1:', met.fbeta_score(test[['is sexual predator']], prediction, 1)

Accuracy:  0.998995338561
Precision:  0.567049808429
Recall: 0.60162601626
F1: 0.58382642998


In [26]:
forest = RandomForestClassifier(n_estimators = 100)

max_f1 = 0
best_features = []

for num_features in range(2, len(column_names)-1):
    for column_name_subset in itertools.combinations(column_names, num_features):

        forest = forest.fit(training[list(column_name_subset)], np.ravel(training.iloc[:,[9]]))
        prediction = forest.predict(test[list(column_name_subset)])

        print column_name_subset
        print 'Accuracy: ', met.accuracy_score(test[['is sexual predator']], prediction) 
        print 'Precision: ', met.precision_score(test[['is sexual predator']], prediction)
        print 'Recall:', met.recall_score(test[['is sexual predator']], prediction)
        print 'F1:', met.f1_score(test[['is sexual predator']], prediction)
        print "\n\n"

        f1 = met.f1_score(test[['is sexual predator']], prediction)
        if max_f1 < f1:
            max_f1 = f1
            best_features =  column_name_subset
                                                                                                 
print max_f1
print best_features

NameError: name 'RandomForestClassifier' is not defined